In [2]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install seaborn --upgrade

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import get_scheduler

In [4]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [5]:
data_path = '/content/IMDB Dataset.csv'
imdb_data = pd.read_csv(data_path)

In [6]:
print("Dataset preview:")
print(imdb_data.head())

Dataset preview:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [7]:
imdb_data['sentiment'] = imdb_data['sentiment'].map({'positive': 1, 'negative': 0})


In [8]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    imdb_data['review'],
    imdb_data['sentiment'],
    test_size=0.2,
    random_state=42
)

In [9]:
print(f"Training set size: {len(train_texts)}")
print(f"Testing set size: {len(test_texts)}")

Training set size: 40000
Testing set size: 10000


In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
train_labels = list(train_labels)
test_labels = list(test_labels)
train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [12]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [15]:
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print("Starting training...")
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{epochs} completed. Average Loss: {total_loss / len(train_loader):.4f}")

print("Training completed.")


Starting training...
Epoch 1/3 completed. Average Loss: 0.2425
Epoch 2/3 completed. Average Loss: 0.1120
Epoch 3/3 completed. Average Loss: 0.0331
Training completed.


In [17]:
model_save_path = "/content/distilbert_imdb_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/distilbert_imdb_model


In [18]:
print("Starting evaluation...")
model.eval()
accuracy = 0

for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = torch.argmax(outputs.logits, dim=-1)
    accuracy += (predictions == batch['labels']).sum().item()

accuracy = accuracy / len(test_dataset)
print(f"Test accuracy: {accuracy:.4f}")


Starting evaluation...
Test accuracy: 0.9337


In [19]:
def predict_sentiment(text):
    model.eval()
    inputs = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    sentiment = "positive" if prediction == 1 else "negative"
    return sentiment


In [20]:
sample_text = "The movie was absolutely wonderful, with great performances!"
predicted_sentiment = predict_sentiment(sample_text)
print(f"Sample Text: {sample_text}")
print(f"Predicted Sentiment: {predicted_sentiment}")

Sample Text: The movie was absolutely wonderful, with great performances!
Predicted Sentiment: positive


In [21]:
import pickle

# Assuming 'model' and 'tokenizer' are your trained DistilBERT objects
model_save_path = "/content/distilbert_imdb_model.pkl"

# Save the model
with open(model_save_path, "wb") as f:
    pickle.dump(model, f)

# Optionally, save the tokenizer as well
tokenizer_save_path = "/content/distilbert_imdb_tokenizer.pkl"
with open(tokenizer_save_path, "wb") as f:
    pickle.dump(tokenizer, f)

print(f"Model saved to {model_save_path}")
print(f"Tokenizer saved to {tokenizer_save_path}")

Model saved to /content/distilbert_imdb_model.pkl
Tokenizer saved to /content/distilbert_imdb_tokenizer.pkl
